In [1]:
from sqlalchemy import create_engine,text
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')
DATABASE_URL=os.getenv('DB_URL')

In [5]:
engine = create_engine(url=DATABASE_URL)

In [6]:
# Test connection
connection = engine.connect()
print(connection)
connection.close()

In [7]:
file_path = './translation.xlsx'
translations_df = pd.read_excel(file_path)

# Convertir los nombres de las columnas a minúsculas
translations_df.columns = [col.lower() for col in translations_df.columns]
translations_df = translations_df.drop_duplicates(subset=['token'], keep='last')


# Verifica que las columnas estén bien definidas
print(translations_df.columns)  
print(translations_df.head())

Index(['token', 'spanish', 'english', 'french', 'arab', 'kreyol', 'swahili'], dtype='object')
                         token  \
0   dizzb_error.invalid_msisdn   
1      dizzb_error.invalid_pin   
2      dizzb_error.pin_expired   
3       dizzb_error.pin_in_use   
4  dizzb_error.request_blocked   

                                             spanish  \
0                   El número ingresado es inválido.   
1                      El PIN ingresado es inválido.   
2  El PIN ya expiró. Se ha enviado uno nuevo. Por...   
3  Ya le enviamos un PIN. chequee en su bandeja d...   
4             Ocurrió un error, vuelva a intentarlo.   

                                             english  \
0                     The number entered is invalid.   
1                        The PIN entered is invalid.   
2  The PIN has expired. A new one has been sent. ...   
3  We have already sent you a PIN. Please check y...   
4               An error occurred, please try again.   

                           

In [59]:
# Obtener los grupos existentes en tr_group
def get_existing_groups():
    with engine.connect() as connection:
        existing_groups = {}
        try:
            result = connection.execute(text("SELECT group_id, group_name FROM tr_group"))
            for row in result:
                existing_groups[row[1]] = row[0]
            return existing_groups
        except Exception as e:
            print("Error al obtener los grupos existentes en tr_group:", e)
            return None

In [60]:
def get_existing_translations():
    with engine.connect() as connection:
        existing_translations = {}
        try:
            result = connection.execute(text("SELECT tr_group_id, language_id, translation_key FROM tr_translation"))
            for row in result:
                existing_translations[(row[0], row[1], row[2])] = True
            return existing_translations
        except Exception as e:
            print("Error al obtener las traducciones existentes en tr_translation:", e)
            return None

In [63]:
existing_groups = get_existing_groups()

# Obtener los prefijos de los tokens del CSV
token_prefixes = set(row['token'].split('.')[0] for _, row in translations_df.iterrows())

# Determinar los nuevos grupos que no están en tr_group
new_groups = token_prefixes - set(existing_groups.keys())
print("Nuevos grupos a insertar:", new_groups)

# Asignar IDs a los nuevos grupos
if existing_groups:
    max_existing_id = max(existing_groups.values())
else:
    max_existing_id = 0

new_group_map = {group: max_existing_id + 1 + i for i, group in enumerate(new_groups)}
print("Mapa de nuevos grupos:", new_group_map)

Nuevos grupos a insertar: set()
Mapa de nuevos grupos: {}


In [62]:
sql_statements_group = []
for group_name, group_id in new_group_map.items():
    sql = f"""
    INSERT INTO tr_group (group_id, group_name, group_enabled, date_created, date_modified)
    VALUES ({group_id}, '{group_name}', 1, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
    """
    sql_statements_group.append(sql)

print(sql_statements_group)

[]


In [65]:
# Ejecutar las consultas
try:
    with engine.connect() as connection:
        for sql in sql_statements_group:
            connection.execute(text(sql))
        connection.commit()
        print("Consultas de inserción ejecutadas con éxito.")
except Exception as e:
    print("Error al ejecutar consultas de inserción:", e)


Consultas de inserción ejecutadas con éxito.


In [66]:
new_group_map.update(existing_groups)
print("Mapa de grupos actualizado:", new_group_map)

Mapa de grupos actualizado: {'general_error': 1, 'opratel_error': 2, 'dizzb_error': 3, 'form_he': 4, 'form_signup': 5, 'form_requestpin': 6, 'form_active': 7, 'form_confirm': 8, 'form_doi': 9, 'rule_error': 10, 'form_error': 11, 'form_ok': 12, 'form_pin': 13}


In [67]:
# Mapeo de language_name a language_id
csv_to_locale = {
    'en': 'english',
    'es': 'spanish',
    'ht' : 'kreyol',
    'sw': 'swahili',
    'ara':'arab',
    'fr': 'french'
}

language_map = {}
sql = """
SELECT language_id, language_locale FROM tr_language;
"""
try:
    with engine.connect() as connection:
        result = connection.execute(text(sql))
        for row in result:
            language_map[row[1]] = row[0]
        print("Mapa de idiomas:", language_map)
except Exception as e:
    print("Error al obtener el mapa de idiomas:", e)
 

Mapa de idiomas: {'es': 1, 'en': 2, 'ht': 3, 'sw': 4, 'ara': 5, 'fr': 6}


In [68]:
import unicodedata
def normalize_and_compare(text1:str, text2:str) -> bool:
    def normalize_text(text:str):
        # Normaliza el texto usando la forma NFC
        normalized = unicodedata.normalize('NFC', text)
        # Elimina los espacios en blanco al inicio y al final
        stripped = normalized.strip()
        # Convierte el texto a minúsculas
        lowercased = stripped.lower()
        return lowercased
    
    normalized_text1 = normalize_text(text1)
    normalized_text2 = normalize_text(text2)
    
    return normalized_text1 == normalized_text2

In [82]:
# Generar consultas SQL para tr_translation verificando la existencia
sql_statements_translation = []
try:
    with engine.connect() as connection:
        for _, row in translations_df.iterrows():
            for locale, column_name in csv_to_locale.items():
                language_id = language_map.get(locale)
                if language_id and not pd.isna(row[column_name]):
                    token_parts = row['token'].split('.')
                    group_name = token_parts[0]
                    translation_item = '.'.join(token_parts[1:])
                    group_id = new_group_map.get(group_name)

                    if group_id:
                        translation_text_raw = row[column_name]
                        translation_text = translation_text_raw.replace("'", "''")
                        # Verificar si la traducción ya existe
                        check_sql = f"""
                        SELECT translation_text FROM tr_translation
                        WHERE language_id = {language_id} AND group_id = {group_id} AND translation_item = '{translation_item}';
                        """
                        result = connection.execute(text(check_sql))
                        existing_translation = result.fetchone()
                        if existing_translation is not None and existing_translation[0] is not None:
                            # Si no existe, agregar la consulta de inserción
                            existing_text = existing_translation[0]

                            if normalize_and_compare(translation_text_raw, existing_text) is False:
                                # print("translation text",translation_text_raw)
                                # print("existing text",existing_text)
                                # print("translation item",translation_item)
                                # print("language id",language_id)
                                # print("group id",group_id)
                                update_sql = f"""
                                UPDATE tr_translation
                                SET translation_text = '{translation_text}', date_modified = CURRENT_TIMESTAMP
                                WHERE language_id = {language_id} AND group_id = {group_id} AND translation_item = '{translation_item}';
                                """
                                sql_statements_translation.append(update_sql)
                        else :
                            insert_sql = f"""
                            INSERT INTO tr_translation (language_id, group_id, translation_item, translation_text, translation_enabled, date_created, date_modified)
                            VALUES ({language_id}, {group_id}, '{translation_item}', '{translation_text}', 1, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
                            """
                            sql_statements_translation.append(insert_sql)
        # Ejecutar las consultas de inserción
        print("Total consultas de inserción en tr_translation:", len(sql_statements_translation))
except Exception as e:
    print("Error al verificar o insertar en tr_translation:", e)

Total consultas de inserción en tr_translation: 220


In [83]:
try:
    with engine.connect() as connection:
        for sql in sql_statements_translation:
            print(f"Ejecutando consulta: {sql}")
            connection.execute(text(sql))
        connection.commit()
        print("Consultas de inserción ejecutadas con éxito.")
except Exception as e:
    print("Error al ejecutar consultas de inserción:", e)

Ejecutando consulta: 
                                UPDATE tr_translation
                                SET translation_text = 'The number entered is invalid.', date_modified = CURRENT_TIMESTAMP
                                WHERE language_id = 2 AND group_id = 3 AND translation_item = 'invalid_msisdn';
                                
Ejecutando consulta: 
                            INSERT INTO tr_translation (language_id, group_id, translation_item, translation_text, translation_enabled, date_created, date_modified)
                            VALUES (3, 3, 'invalid_msisdn', 'Nimewo antre a valab.', 1, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP);
                            
Ejecutando consulta: 
                            INSERT INTO tr_translation (language_id, group_id, translation_item, translation_text, translation_enabled, date_created, date_modified)
                            VALUES (4, 3, 'invalid_msisdn', 'Nambari uliyoweka si sahihi.', 1, CURRENT_TIMESTAMP, CURRENT_TIME